# Hydrometeor Classification from Polarimetric Radar Data

- Reading radar data with [`pyart`](https://github.com/ARM-DOE/pyart)
    - FCTH (S Band, Dual Pol) - variables *corrected_reflectivity*, *cross_correlation_ratio*, *differential_reflectivity*, *specific_differential_phase*
    - Cases:
        - 2016-12-25
        - 2017-01-31
        - 2017-03-14
        - 2017-11-15
        - 2017-11-16
  
  
- Processing data with [`csu_radartools`](https://github.com/CSU-RadarMet/CSU_RadarTools)
    - Classifying into 10 hydrometeor types (Drizzle, Rain, Ice Crystals, Aggregates, Wet/Melting Snow, Vertically Aligned Ice, Low-Density Graupel, High-Density Graupel, Hail and Big Drops)
    - Calculating liquid and ice water mass
    - Plotting data

Based on [CSU_RadarTools Demonstration](https://github.com/CSU-Radarmet/CSU_RadarTools/blob/master/notebooks/CSU_RadarTools_Demo.ipynb).

## Loading necessary packages

In [ ]:
from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import glob
from datetime import datetime

import read_brazil_radar as rbr
import pyart
from skewt import SkewT
from csu_radartools import (csu_fhc, csu_liquid_ice_mass, csu_blended_rain, 
                            csu_dsd, csu_kdp, csu_misc, fundamentals)
%matplotlib inline

## Defining necessary functions

### `interpolate_sounding_to_radar`
Interpolating sounding data to radar

In [ ]:
def interpolate_sounding_to_radar(sounding, radar):
    radar_z = get_z_from_radar(radar)
    radar_T = None
    snd_T, snd_z = check_sounding_for_montonic(sounding)
    shape = np.shape(radar_z)
    rad_z1d = radar_z.ravel()
    rad_T1d = np.interp(rad_z1d, snd_z, snd_T)
    return np.reshape(rad_T1d, shape), radar_z

### `add_field_to_radar_object`
Transforming array into radar data

In [ ]:
def add_field_to_radar_object(field, radar, field_name='FH', units='unitless', 
                              long_name='Hydrometeor ID', standard_name='Hydrometeor ID',
                              dz_field='corrected_reflectivity'):
    """
    Adds a newly created field to the Py-ART radar object. If reflectivity is a masked array,
    make the new field masked the same as reflectivity.
    """
    fill_value = -32768
    masked_field = np.ma.asanyarray(field)
    masked_field.mask = masked_field == fill_value
    if hasattr(radar.fields[dz_field]['data'], 'mask'):
        setattr(masked_field, 'mask', 
                np.logical_or(masked_field.mask, radar.fields[dz_field]['data'].mask))
        fill_value = radar.fields[dz_field]['data'].fill_value
    field_dict = {'data': masked_field,
                  'units': units,
                  'long_name': long_name,
                  'standard_name': standard_name,
                  'fill_value': fill_value}
    radar.add_field(field_name, field_dict, replace_existing=True)
    return radar

### `radar_coords_to_cart`
Converting radar to cartesian coordinates

In [ ]:
def radar_coords_to_cart(rng, az, ele, debug=False):
    """
    TJL - taken from old Py-ART version
    Calculate Cartesian coordinate from radar coordinates
    Parameters
    ----------
    rng : array
        Distances to the center of the radar gates (bins) in kilometers.
    az : array
        Azimuth angle of the radar in degrees.
    ele : array
        Elevation angle of the radar in degrees.
    Returns
    -------
    x, y, z : array
        Cartesian coordinates in meters from the radar.
    Notes
    -----
    The calculation for Cartesian coordinate is adapted from equations
    2.28(b) and 2.28(c) of Doviak and Zrnic [1]_ assuming a
    standard atmosphere (4/3 Earth's radius model).
    .. math::
        z = \\sqrt{r^2+R^2+r*R*sin(\\theta_e)} - R
        s = R * arcsin(\\frac{r*cos(\\theta_e)}{R+z})
        x = s * sin(\\theta_a)
        y = s * cos(\\theta_a)
    Where r is the distance from the radar to the center of the gate,
    :math:\\theta_a is the azimuth angle, :math:\\theta_e is the
    elevation angle, s is the arc length, and R is the effective radius
    of the earth, taken to be 4/3 the mean radius of earth (6371 km).
    References
    ----------
    .. [1] Doviak and Zrnic, Doppler Radar and Weather Observations, Second
        Edition, 1993, p. 21.
    """
    theta_e = ele * np.pi / 180.0    #-- elevation angle in radians
    theta_a = az * np.pi / 180.0     #-- azimuth angle in radians
    R = 6371.0 * 1000.0 * 4.0 / 3.0  #-- effective radius of earth in meters
    r = rng * 1000.0                 #-- distances to gates in meters

    z = (r ** 2 + R ** 2 + 2.0 * r * R * np.sin(theta_e)) ** 0.5 - R
    s = R * np.arcsin(r * np.cos(theta_e) / (R + z))  #-- arc length in m
    x = s * np.sin(theta_a)
    y = s * np.cos(theta_a)
    return x, y, z

### `get_z_from_radar`
Calculating radar height correspondent to elevations

In [ ]:
def get_z_from_radar(radar):
    azimuth_1D = radar.azimuth['data']
    elevation_1D = radar.elevation['data']
    srange_1D = radar.range['data']
    sr_2d, az_2d = np.meshgrid(srange_1D, azimuth_1D)
    el_2d = np.meshgrid(srange_1D, elevation_1D)[1]
    xx, yy, zz = radar_coords_to_cart(sr_2d/1000.0, az_2d, el_2d)
    return zz + radar.altitude['data']

### `check_sounding_for_montonic`
Forcing sounding data to be monotonic

In [ ]:
def check_sounding_for_montonic(sounding):
    snd_T = sounding.soundingdata['temp']  # In old SkewT, was sounding.data
    snd_z = sounding.soundingdata['hght']  # In old SkewT, was sounding.data
    dummy_z = []
    dummy_T = []
    if not snd_T.mask[0]: #May cause issue for specific soundings
        dummy_z.append(snd_z[0])
        dummy_T.append(snd_T[0])
        for i, height in enumerate(snd_z):
            if i > 0:
                if snd_z[i] > snd_z[i-1] and not snd_T.mask[i]:
                    dummy_z.append(snd_z[i])
                    dummy_T.append(snd_T[i])
        snd_z = np.array(dummy_z)
        snd_T = np.array(dummy_T)
    return snd_T, snd_z

### `adjust_fhc_colorbar_for_pyart`, `adjust_meth_colorbar_for_pyart`
Create custom colorbar for HID plots

In [ ]:
hid_colors = ['White', 'LightBlue', 'MediumBlue', 'DarkOrange', 'LightPink',
              'Cyan', 'DarkGray', 'Lime', 'Yellow', 'Red', 'Fuchsia']
cmaphid = colors.ListedColormap(hid_colors)
cmapmeth = colors.ListedColormap(hid_colors[0:6])
cmapmeth_trop = colors.ListedColormap(hid_colors[0:7])

def adjust_fhc_colorbar_for_pyart(cb):
    cb.set_ticks(np.arange(1.4, 10, 0.9))
    cb.ax.set_yticklabels(['Drizzle', 'Rain', 'Ice Crystals', 'Aggregates',
                           'Wet Snow', 'Vertical Ice', 'LD Graupel',
                           'HD Graupel', 'Hail', 'Big Drops'])
    cb.ax.set_ylabel('')
    cb.ax.tick_params(length=0)
    return cb

def adjust_meth_colorbar_for_pyart(cb, tropical=False):
    if not tropical:
        cb.set_ticks(np.arange(1.25, 5, 0.833))
        cb.ax.set_yticklabels(['R(Kdp, Zdr)', 'R(Kdp)', 'R(Z, Zdr)', 'R(Z)', 'R(Zrain)'])
    else:
        cb.set_ticks(np.arange(1.3, 6, 0.85))
        cb.ax.set_yticklabels(['R(Kdp, Zdr)', 'R(Kdp)', 'R(Z, Zdr)', 'R(Z_all)', 'R(Z_c)', 'R(Z_s)'])
    cb.ax.set_ylabel('')
    cb.ax.tick_params(length=0)
    return cb

### `read_calculate_radar`
Using radar and sounding filenames, read and calculate:
- Temperature and height profiles
- Hydrometeor classification
- Liquid and ice masses, ice fraction

In [ ]:
def read_calculate_radar(filename, s_names):
    #-- Reading data
    file = rbr.read_rainbow_hdf5(filename)
    file_date = pd.to_datetime(file.time['units'][14:])
    
    #-- Interpolating with sounding
    sounding = SkewT.Sounding(s_names.loc[str(file_date.date())].item())
    file_T, file_z = interpolate_sounding_to_radar(sounding, file)
    
    #-- Extracting necessary variables
    z_corrected = file.fields['corrected_reflectivity']['data']
    zdr = file.fields['differential_reflectivity']['data']
    kdp = file.fields['specific_differential_phase']['data']
    rho_hv = file.fields['cross_correlation_ratio']['data']
    
    #-- Classifying
    scores = csu_fhc.csu_fhc_summer(dz=z_corrected, zdr=zdr, kdp=kdp, rho=rho_hv, 
                                    use_temp=True, T=file_T, band='S')
    fh = np.argmax(scores, axis = 0) + 1
    #--- Adding to radar file
    file = add_field_to_radar_object(fh, file)
    
    #-- Calculating liquid and ice mass
    mw, mi = csu_liquid_ice_mass.calc_liquid_ice_mass(z_corrected, zdr, file_z/1000.0, T=file_T)
    fi = mi / (mw + mi)
    
    #--- Adding to radar file
    file = add_field_to_radar_object(mw, file, field_name='MW', units='g m-3', long_name='Liquid Water Mass', 
                                     standard_name='Liquid Water Mass')
    file = add_field_to_radar_object(mi, file, field_name='MI', units='g m-3', long_name='Ice Water Mass', 
                                     standard_name='Ice Water Mass')
    file = add_field_to_radar_object(fi, file, field_name='FI', units='dimensionless', long_name='Ice Mass Fraction', 
                                     standard_name='Ice Mass Fraction')

    print('File ' + filename + ' read!')
    return file

### `plot_horizontal_panel`
Using radar and hailpads data, plot horizontal view of polarimetric fields with hailpad position

In [ ]:
def plot_horizontal_panel(file, hailpads, display_limits, shapefile, radar, save_to, sweep=0):
    #-- Defining date, sweep and hailpad data for the date
    file_date = pd.to_datetime(file.time['units'][14:])
    file_sweep = file.fixed_angle['data'][sweep]
    hailpad = hailpads.loc[file_date.strftime('%Y-%m-%d')]
        
    #-- Start to plot
    display = pyart.graph.RadarMapDisplay(file)
    fig, ax = plt.subplots(3, 2, sharex=True, sharey=True, figsize=[8.5,8])
    fig.set_facecolor('w')
    xlim, ylim = display_limits
    
    #--- Reflectivity
    display.plot_ppi_map('corrected_reflectivity', sweep=sweep, ax=ax[0,0], shapefile=shapefile, 
                         max_lat=ylim[1], min_lat=ylim[0], min_lon=xlim[0], max_lon=xlim[1],
                         vmin = 10, vmax = 70, mask_outside=True, title_flag=False, 
                         lat_lines=np.arange(ylim[0], ylim[1], .2), lon_lines=np.arange(xlim[0], xlim[1], .25))
    display.plot_point(lat=hailpad['lat'], lon=hailpad['lon'], ax=ax[0,0], symbol = 'ks', #-- Hailpad
                       markersize=10, alpha=0.9, markerfacecolor='snow')
    
    #--- ZDR
    display.plot_ppi_map('differential_reflectivity', sweep=sweep, ax=ax[0,1], shapefile=shapefile, 
                         max_lat=ylim[1], min_lat=ylim[0], min_lon=xlim[0], max_lon=xlim[1],
                         vmin = -2, vmax = 7, mask_outside=True, title_flag=False,
                         lat_lines=np.arange(ylim[0], ylim[1], .2), lon_lines=np.arange(xlim[0], xlim[1], .25))
    display.plot_point(lat=hailpad['lat'], lon=hailpad['lon'], ax=ax[0,1], symbol = 'ks', #-- Hailpad
                       markersize=10, alpha=0.9, markerfacecolor='snow')
    
    #--- KDP
    display.plot_ppi_map('specific_differential_phase', sweep=sweep, ax=ax[1,0], shapefile=shapefile, 
                         max_lat=ylim[1], min_lat=ylim[0], min_lon=xlim[0], max_lon=xlim[1],
                         vmin = -1, vmax = 2, mask_outside=True, title_flag=False, 
                         lat_lines=np.arange(ylim[0], ylim[1], .2), lon_lines=np.arange(xlim[0], xlim[1], .25))
    display.plot_point(lat=hailpad['lat'], lon=hailpad['lon'], ax=ax[1,0], symbol = 'ks', #-- Hailpad
                       markersize=10, alpha=0.9, markerfacecolor='snow')

    #--- RHO
    display.plot_ppi_map('cross_correlation_ratio', sweep=sweep, ax=ax[1,1], shapefile=shapefile, 
                         max_lat=ylim[1], min_lat=ylim[0], min_lon=xlim[0], max_lon=xlim[1],
                         vmin = 0.9, vmax = 1, mask_outside=True, title_flag=False, 
                         lat_lines=np.arange(ylim[0], ylim[1], .2), lon_lines=np.arange(xlim[0], xlim[1], .25))
    display.plot_point(lat=hailpad['lat'], lon=hailpad['lon'], ax=ax[1,1], symbol = 'ks', #-- Hailpad
                       markersize=10, alpha=0.9, markerfacecolor='snow')
    
    #--- Ice Fraction
    display.plot_ppi_map('FI', sweep=sweep, ax=ax[2,0], shapefile=shapefile, 
                         max_lat=ylim[1], min_lat=ylim[0], min_lon=xlim[0], max_lon=xlim[1],
                         vmin = 0, vmax = 1, mask_outside=True, title_flag=False, cmap='YlGnBu',
                         lat_lines=np.arange(ylim[0], ylim[1], .2), lon_lines=np.arange(xlim[0], xlim[1], .25))
    display.plot_point(lat=hailpad['lat'], lon=hailpad['lon'], ax=ax[2,0], symbol = 'ks', #-- Hailpad
                       markersize=10, alpha=0.9, markerfacecolor='snow')

    #--- Hydrometeor Classification
    display.plot_ppi_map('FH', sweep=sweep, ax=ax[2,1], shapefile=shapefile, 
                         max_lat=ylim[1], min_lat=ylim[0], min_lon=xlim[0], max_lon=xlim[1],
                         vmin = 0, vmax = 10, mask_outside=True, title_flag=False, cmap=cmaphid, 
                         lat_lines=np.arange(ylim[0], ylim[1], .2), lon_lines=np.arange(xlim[0], xlim[1], .25))
    display.plot_point(lat=hailpad['lat'], lon=hailpad['lon'], ax=ax[2,1], symbol = 'ks', #-- Hailpad
                       markersize=10, alpha=0.9, markerfacecolor='snow')
    
    #--- Common aspects
    plt.suptitle(radar.upper() + ' '  + str(file_sweep) + ' deg ' + str(file_date) + ' UTC', 
                 weight='bold', stretch='condensed', size='xx-large', y=0.92)
    display.cbs[5] = adjust_fhc_colorbar_for_pyart(display.cbs[5])
    
    #-- Saving the figure
    plt.savefig(save_to + radar + '_ppi_h_' + file_date.strftime('%Y%m%d%H%M') + 'UTC.png', 
                dpi=300, transparent=True, bbox_inches='tight')
    plt.close()
    
    return 'Plotting horizontal view for date ' + str(file_date)

### `plot_vertical_panel`
Using radar and hailpads data, plot vertical view of polarimetric fields (only when hailfall occurred, defined by *hailpads_data*) with hailpads position

In [ ]:
def plot_vertical_panel(file, hailpads, x_limits, radar, save_to):
    #-- Defining date and hailpad data for the date
    file_date = pd.to_datetime(file.time['units'][14:])
    hailpad = hailpads.loc[file_date.strftime('%Y-%m-%d')]
    
    #-- Plotting only when hailfall occurred
    if file_date.strftime('%H-%M') == hailpad['time_cth']:
        #--- Selecting cross-section
        file_cs = pyart.util.cross_section_ppi(file, [hailpad['azim_cth']])
        
        #--- Start to plot
        display = pyart.graph.RadarDisplay(file_cs)
        fig, ax = plt.subplots(4, 2, sharex=True, sharey=True, figsize=[8.5,10])
        fig.set_facecolor('w')
   
        #---- Reflectivity
        display.plot('corrected_reflectivity', vmin=10, vmax=70, title='', ax=ax[0,0], axislabels_flag=False)
        pnt = ax[0,0].scatter(x=hailpad['pos_cth'], y=1, marker='s', c='snow', alpha=0.9, edgecolors='black') #-- Hailpad
        ax[0,0].grid(linestyle=':', c='k')
        
        #---- ZDR
        display.plot('differential_reflectivity', vmin=-2, vmax=7, title='', ax=ax[0,1], axislabels_flag=False)
        pnt = ax[0,1].scatter(x=hailpad['pos_cth'], y=1, marker='s', c='snow', alpha=0.9, edgecolors='black') #-- Hailpad
        ax[0,1].grid(linestyle=':', c='k')
         
        #---- KDP
        display.plot('specific_differential_phase', vmin=-1, vmax=2, title='', ax=ax[1,0], axislabels_flag=False)
        pnt = ax[1,0].scatter(x=hailpad['pos_cth'], y=1, marker='s', c='snow', alpha=0.9, edgecolors='black') #-- Hailpad
        ax[1,0].grid(linestyle=':', c='k')

        #---- RHO
        display.plot('cross_correlation_ratio', vmin=0.9, vmax=1, title='', ax=ax[1,1], axislabels_flag=False)
        pnt = ax[1,1].scatter(x=hailpad['pos_cth'], y=1, marker='s', c='snow', alpha=0.9, edgecolors='black') #-- Hailpad
        ax[1,1].grid(linestyle=':', c='k')

        #---- Hydrometeor Classification
        display.plot('FH', vmin=0, vmax=10, cmap=cmaphid, title = '', ax=ax[2,0], axislabels_flag=False)
        pnt = ax[2,0].scatter(x=hailpad['pos_cth'], y=1, marker='s', c='snow', alpha=0.9, edgecolors='black') #-- Hailpad
        ax[2,0].grid(linestyle=':', c='k')

        #---- Doppler Velocity
        display.plot('velocity', vmin=-15, vmax=15, title='', ax=ax[2,1], axislabels_flag=False)
        pnt = ax[2,1].scatter(x=hailpad['pos_cth'], y=1, marker='s', c='snow', alpha=0.9, edgecolors='black') #-- Hailpad
        ax[2,1].grid(linestyle=':', c='k')

        #---- Liquid Water Mass
        display.plot('MW', vmin = 0, vmax = 15, cmap='YlGnBu', title = '', ax=ax[3,0], axislabels_flag=False)
        pnt = ax[3,0].scatter(x=hailpad['pos_cth'], y=1, marker='s', c='snow', alpha=0.9, edgecolors='black') #-- Hailpad
        ax[3,0].grid(linestyle=':', c='k')

        #---- Ice Water Mass
        display.plot('MI', vmin = 0, vmax = 15, cmap='YlGnBu', title = '', ax=ax[3,1], axislabels_flag=False)
        pnt = ax[3,1].scatter(x=hailpad['pos_cth'], y=1, marker='s', c='snow', alpha=0.9, edgecolors='black') #-- Hailpad
        ax[3,1].grid(linestyle=':', c='k')
        
        #---- Common aspects
        plt.suptitle(radar.upper() + ' ' + str(file_date) + ' UTC - Azimuth = ' + str(hailpad['azim_cth']) + ' deg', 
                     weight='bold', stretch='condensed', size='xx-large', y=0.92)
        plt.xlim(x_limits)
        plt.ylim((0,18))
        
        #---- Manual labelling (outer_label doesn't work!)
        ax[0,0].set(xlabel='', ylabel='Distance above\nRadar (km)')
        ax[0,1].set(xlabel='', ylabel='')
        ax[1,0].set(xlabel='', ylabel='Distance above\nRadar (km)')
        ax[1,1].set(xlabel='', ylabel='')
        ax[2,0].set(xlabel='', ylabel='Distance above\nRadar (km)')
        ax[2,1].set(xlabel='', ylabel='')
        ax[3,0].set(xlabel='Distance from Radar (km)', ylabel='Distance above\nRadar (km)')
        ax[3,1].set(xlabel='Distance from Radar (km)', ylabel='')        
        
        display.cbs[4] = adjust_fhc_colorbar_for_pyart(display.cbs[4])
        
        #-- Saving the figure
        plt.savefig(save_to + radar + '_ppi_v_' + file_date.strftime('%Y%m%d%H%M') + 'UTC.png', 
                    dpi=300, transparent=True, bbox_inches='tight')
        plt.close()
        
    return 'Plotting vertical view for date ' + str(file_date)

## Defining filepaths and custom variables

In [ ]:
cth_filenames = open("files_cth_level0.txt").read().split('\n')
hailpad_data = pd.read_csv("../Data/GENERAL/hailpads_registry.txt", index_col=0)
sounding_filenames = pd.read_csv("soundings_registry.txt", index_col=0)
shapefile = "../Data/GENERAL/shapefiles/sao_paulo"
save_path = "figures/ppis/classification/"
radar_name = "fcth"

#-- Limits in horizontal view panel
#--- [[xlim], [ylim]]
limits = [[-47.5,-46.5], [-23.3,-22.5]]

#-- Limits (x axis) in vertical panel
#--- (xlim)
xlimits = (130,160)

print(hailpad_data)
print(sounding_filenames)

## Processing all the files in *filenames_cth*

In [ ]:
for filename in cth_filenames:
    radar = read_calculate_radar(filename, sounding_filenames)
    print(plot_horizontal_panel(radar, hailpad_data, limits, shapefile, radar_name, save_path))
    print(plot_vertical_panel(radar, hailpad_data, xlimits, radar_name, save_path))